In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as fun

#### Using the walmart_stock.csv file

#### Starting a simple Spark Session

In [2]:
spark = SparkSession.builder.getOrCreate()

#### Loading the Walmart Stock CSV File

In [3]:
data = spark.read.csv('walmart_stock.csv', header=True)

#### What are the column names?

In [4]:
data.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [5]:
data.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



#### Print out the first 5 columns.

In [6]:
data.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



#### Use describe() to learn about the DataFrame.

In [7]:
data.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|         10010500|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

#### Casting Some columns to different datatype

In [8]:
data = data.withColumn("Date", data["Date"].cast('timestamp'))
data = data.withColumn("Open", data["Open"].cast('float'))
data = data.withColumn("High", data["High"].cast('float'))
data = data.withColumn("Low", data["Low"].cast('float'))
data = data.withColumn("Close", data["Close"].cast('float'))
data = data.withColumn("Volume", data["Volume"].cast('float'))
data = data.withColumn("Adj Close", data["Adj Close"].cast('float'))

In [9]:
data.show(5)

+-------------------+-----+-----+-----+-----+---------+---------+
|               Date| Open| High|  Low|Close|   Volume|Adj Close|
+-------------------+-----+-----+-----+-----+---------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|1.26688E7|52.619236|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71|9593300.0|52.078476|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|1.27682E7| 51.82554|
|2012-01-06 00:00:00|59.42|59.45|58.87| 59.0|8069400.0| 51.45922|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18|6679300.0|51.616215|
+-------------------+-----+-----+-----+-----+---------+---------+
only showing top 5 rows



In [10]:
data.summary().show()

+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|             Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|72.35785375452572| 72.8393880756178|71.91860094964979|72.38844997363553|8222093.478537361|67.23883840200064|
| stddev|6.768090251767697|6.768186825250206|6.744075739203606|6.756859160119612|4519780.791987604|6.722609385249684|
|    min|            56.39|            57.06|             56.3|            56.42|        2094900.0|         50.36369|
|    25%|            68.62|            69.06|            68.16|            68.63|        5790200.0|        63.775513|
|    50%|            73.23|            73.72|           

#### Creating a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [11]:
new_data = data.withColumn("HV Ratio", data["High"]/data["Volume"])
new_data.select('HV Ratio').show(5)

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714682786927E-6|
|6.290848662516662E-6|
| 4.66941298944916E-6|
| 7.36733843444859E-6|
|8.915604814435727E-6|
+--------------------+
only showing top 5 rows



#### What day had the Peak High in Price?

In [12]:
data.sort(fun.col('High'), ascending=False).select("Date").show(1)

+-------------------+
|               Date|
+-------------------+
|2015-01-13 00:00:00|
+-------------------+
only showing top 1 row



#### What is the mean of the Close column?

In [13]:
data.select(fun.mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844997363553|
+-----------------+



#### What is the max and min of the Volume column?

In [14]:
data.select(fun.max('Volume')).show()

+-----------+
|max(Volume)|
+-----------+
|8.0898096E7|
+-----------+



In [15]:
data.select(fun.min('Volume')).show()

+-----------+
|min(Volume)|
+-----------+
|  2094900.0|
+-----------+



#### How many days was the Close lower than 60 dollars?

In [16]:
new_data.select('Close').where(new_data['Close']<60).agg(fun.count(new_data['Close'])).show()

+------------+
|count(Close)|
+------------+
|          81|
+------------+



#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [17]:
new_data.filter(new_data['High'] > 80).count() / new_data.select('High').count()

0.09141494435612083

#### What is the Pearson correlation between High and Volume?

In [18]:
new_data.stat.corr('High','Volume')

-0.3384326095027024

#### What is the max High per year?

In [19]:
new_data.groupBy(fun.year('Date')).agg(fun.max('High')).show()

+----------+---------+
|year(Date)|max(High)|
+----------+---------+
|      2015|    90.97|
|      2013|    81.37|
|      2014|    88.09|
|      2012|     77.6|
|      2016|    75.19|
+----------+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc

In [20]:
new_data.groupBy(fun.month('Date')).agg(fun.mean('Close')).show()

+-----------+-----------------+
|month(Date)|       avg(Close)|
+-----------+-----------------+
|         12|72.84792482628012|
|          1| 71.4480196131338|
|          6| 72.4953774506191|
|          3|71.77794376266337|
|          5|72.30971685445533|
|          9|72.18411782208611|
|          4|72.97361900692894|
|          8| 73.0298185521906|
|          7|74.43971944078106|
|         10| 71.5785454489968|
|         11|72.11108927207418|
|          2|71.30680438169499|
+-----------+-----------------+

